Wstęp do sztucznej inteligencji. Lista 3. Zadanie 3
Karol Janic. Maj 2023. **Sieć neuronowa przyjmująca 2 liczby z zakresu [-1, 1] różne od 0 zwracająca 1 gdy mają ten sam znak albo jeden gdy mają różne znaki**. Sieć ma 2 warstwy, warstwa ukryta ma 4 neurony a wyjściowa jeden. **Implementacja algorytmu propagacji wstecznej.**

In [343]:
import random
from math import exp

Implementacja funkcji aktywacji ReLU

In [344]:
def ReLU(x):
  return max(0, x)

Implementacja funkcji aktywacji Sigmoid

In [345]:
def Sigmoid(x):
 try:
    result = 1.0 / (1.0 + exp(-x))
 except OverflowError:
    result = float('inf')
 return result

Implementacja funkcji aktywacji neuronu

In [346]:
def activate(weights, inputs):
  result = weights[-1]

  for i in range(len(weights) - 1):
    result += weights[i] * inputs[i]
  return result

Implementacja pochodnej

In [347]:
def transfer_derivative(x, activation_function):
  if activation_function == 'relu':
    return 0 if x < 0 else 1
  elif activation_function == 'sigmoid':
    return x * (1.0 - x)
  else:
    raise Exception('Invalid param: activation_function')


Implementacja propagacji do przodu

In [348]:
def forward_propagate(network, row, activation_function):
  inputs = row
  for layer in network:
    new_inputs = []
    for neuron in layer:
      activation = activate(neuron['weights'], inputs)
      
      if activation_function == 'sigmoid':
        neuron['output'] = Sigmoid(activation)
      elif activation_function == 'relu':
        neuron['output'] = ReLU(activation)
      else:
        raise Exception('Invalid param: activation_function')

      new_inputs.append(neuron['output'])
    inputs = new_inputs

  return inputs

Implementacja propagacji błędów

In [349]:
def backward_propagate_error(network, expected, normalization_method, activation_function):
  for i in reversed(range(len(network))):
    layer = network[i]
    errors = list()
    if i != len(network) - 1:
      for j in range(len(layer)):
        error = 0.0
        for neuron in network[i + 1]:
          error += (neuron['weights'][j] * neuron['delta'])
        errors.append(error)
    else:
      for j in range(len(layer)):
        neuron = layer[j]
        error = neuron['output'] - expected[j]

        norm_lambda1 = 0.001
        norm_lambda2 = 0.0001

        l1_norm = sum(sum(sum(abs(weight) for weight in neuron['weights']) for neuron in layer) for layer in network)
        l2_norm = sum(sum(sum(weight**2 for weight in neuron['weights']) for neuron in layer) for layer in network)

        if normalization_method == 'any':
          error += 0
        elif normalization_method == 'l1':
          error += norm_lambda1 * l1_norm
        elif normalization_method == 'l2':
          error += norm_lambda2 * l2_norm
        else:
          raise Exception('Invalid param: normalization_method')

        errors.append(error)

    for j in range(len(layer)):
      neuron = layer[j]
      neuron['delta'] = errors[j] * transfer_derivative(neuron['output'], activation_function)

Implementacja funkcji aktualizującej wagi na podstawie błędu

In [350]:
def update_weights(network, row, l_rate):
  for i in range(len(network)):
    inputs = row[:-1]
    if i != 0:
      inputs = [neuron['output'] for neuron in network[i - 1]]
    for neuron in network[i]:
      for j in range(len(inputs)):
        neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
      neuron['weights'][-1] -= l_rate * neuron['delta']

Funkcja inicjalizująca sieć

In [351]:
def initialize_network(n_inputs, n_hidden, n_outputs):
  network = list()
  hidden_layer = [{'weights':[random.random() for i in range(n_inputs + 1)], 'delta': 1} for i in range(n_hidden)]
  network.append(hidden_layer)
  output_layer = [{'weights':[random.random() for i in range(n_hidden + 1)], 'delta': 1} for i in range(n_outputs)]
  network.append(output_layer)
  return network

Funkcja trenująca sieć

In [352]:
def train_network(network, train, l_rate, n_epoch, n_outputs, activation_function, normalization_method):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs = forward_propagate(network, row, activation_function)
      expected = [0 for i in range(n_outputs)]
      expected[row[-1]] = 1
      
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])

      backward_propagate_error(network, expected, normalization_method, activation_function)
      update_weights(network, row, l_rate)
    print(f'epoch={epoch}, lrate={l_rate}, error={sum_error}')

Funkcja predykcji sieci

In [353]:
def predict(network, row, activation_function):
  outputs = forward_propagate(network, row, activation_function)
  return outputs.index(max(outputs))

Implementacja funkcji oceniającej jakość algorytmu (liczba poprawnie przewidzianych wartości)

In [354]:
def accuracy_metric(actual, predicted):
  correct = 0

  for i in range(len(actual)):
    if actual[i] == predicted[i]:
      correct += 1

  return correct / float(len(actual)) * 100.0

**Implementacja algorytmu propagacji wstecznej ze stochastycznym spadkiem gradientu**

In [355]:
def back_propagation(train, test, activation_function, normalization_method, l_rate, n_epoch, n_hidden):
  n_inputs = len(train[0]) - 1
  n_outputs = len(set([row[-1] for row in train]))
  print(n_outputs)

  network = initialize_network(n_inputs, n_hidden, n_outputs)
  train_network(network, train, l_rate, n_epoch, n_outputs, activation_function, normalization_method)
  predictions = list()

  for row in test:
    prediction = predict(network, row, activation_function)
    predictions.append(prediction)

  return(predictions)

Implementacja podziału danych do walidacji krzyżowej

In [356]:
def cross_validation_split(dataset, n_folds):
  dataset_split = list()
  dataset_copy = list(dataset)

  fold_size = int(len(dataset) / n_folds)

  for i in range(n_folds):
    fold = list()

    while len(fold) < fold_size:
      index = random.randrange(len(dataset_copy))
      fold.append(dataset_copy.pop(index))

    dataset_split.append(fold)

  return dataset_split

Implementacja funkcji ewaluującej sieć

In [357]:
def evaluate_algorithm(dataset, n_folds, activation_function, normalization_method, *args):
  folds = cross_validation_split(dataset, n_folds)
  scores = list()

  for fold in folds:
    train_set = list(folds)
    train_set.remove(fold)
    train_set = sum(train_set, [])
    test_set = list()

    for row in fold:
      row_copy = list(row)
      test_set.append(row_copy)
      row_copy[-1] = None

    predicted = back_propagation(train_set, test_set, activation_function, normalization_method, *args)
    actual = [row[-1] for row in fold]
    accuracy = accuracy_metric(actual, predicted)
    scores.append(accuracy)

  return scores

Funkcja generująca zbiór danych

In [358]:
def create_dataset(size):
  dataset = []
  for _ in range(size):
    x, y = 0,0
    while x == 0 or y == 0:
      x, y = random.uniform(-1, 1), random.uniform(-1, 1)

    sample = (x, y, 1 if x * y > 0 else 0)
    dataset.append(sample)

  return dataset

Testowanie sieci

In [374]:
n_folds = 5
l_rate = 0.1
n_epoch = 10
n_hidden = 4

activation_function = 'sigmoid'
normalization_method = 'l2'
normalization_lambda = 0.001

dataset = create_dataset(10000)

scores = evaluate_algorithm(dataset, n_folds, activation_function, normalization_method, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

2
epoch=0, lrate=0.1, error=4018.0902365095844
epoch=1, lrate=0.1, error=3125.274839561621
epoch=2, lrate=0.1, error=1949.0927993564283
epoch=3, lrate=0.1, error=1856.4248599365842
epoch=4, lrate=0.1, error=1843.085423591014
epoch=5, lrate=0.1, error=1821.897226863523
epoch=6, lrate=0.1, error=1692.983459624626
epoch=7, lrate=0.1, error=1432.4288656815027
epoch=8, lrate=0.1, error=1175.7071900789483
epoch=9, lrate=0.1, error=1044.8227218318734
2
epoch=0, lrate=0.1, error=3999.974138000627
epoch=1, lrate=0.1, error=3035.6212908333737
epoch=2, lrate=0.1, error=1966.2281543126107
epoch=3, lrate=0.1, error=1883.381388905978
epoch=4, lrate=0.1, error=1853.2629444505828
epoch=5, lrate=0.1, error=1705.6449762413033
epoch=6, lrate=0.1, error=1417.290039682066
epoch=7, lrate=0.1, error=1165.461836357352
epoch=8, lrate=0.1, error=1051.375995020225
epoch=9, lrate=0.1, error=1001.6263331899016
2
epoch=0, lrate=0.1, error=3915.508710729411
epoch=1, lrate=0.1, error=2581.6637088141165
epoch=2, lrate

Testy przeprowadzono na podstawie 10000 losowo wygenerownaych próbek. Sieć była trenowana przez 10 epok.

Mean Accuracy sieci dla funkcji aktywacji sigmoid(Współczynnik uczenia = 0.1, Współczynnik normalizacji 0.001):

*   brak normalizacji: 93%
*   normalizacja L1: 89%
*   normalizacja L2: 91%


Mean Accuracy sieci dla funkcji aktywacji relu(Współczynnik uczenia = 0.01, Współczynnik normalizacji 0.0001):

*   brak normalizacji: 70%
*   normalizacja L1: 73%
*   normalizacja L2: 71%

W przypadku funkcji aktywacji sigmoid wartość błędu maleje szybciej

